In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
import pickle
import os

In [2]:
flight_data = (
    pd.read_csv('final_dataset_cleaned_v1.csv')
)

In [3]:
flight_data.shape

(12658696, 29)

In [4]:
flight_data.head(n = 3)

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,TAIL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,...,time,temp,dwpt,rhum,prcp,wdir,wspd,pres,coco,target
0,2020,1,1,1,3,2020-01-01,N951WN,ONT,"Ontario, CA",SFO,...,2020-01-01 18:00:00,12.2,-7.3,25.0,0.0,330.0,18.4,1017.3,3.0,1
1,2020,1,1,1,3,2020-01-01,N467WN,ONT,"Ontario, CA",SFO,...,2020-01-01 11:00:00,3.9,-3.4,59.0,0.0,210.0,5.4,1017.6,2.0,0
2,2020,1,1,1,3,2020-01-01,N7885A,ONT,"Ontario, CA",SJC,...,2020-01-01 20:00:00,12.8,-7.3,24.0,0.0,290.0,11.2,1015.8,2.0,0


In [3]:
flight_data['aircraft_type_new'] = [i.split('-')[0] for i in flight_data['Aircraft type']]

In [4]:
# list of numerical and categorical features to select 
l_numeric_cols = [ 'DEP_HOUR', 'temp', 'dwpt', 'rhum', 'prcp', 'wspd', 'pres', 'coco']
l_categorical_cols = ['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'aircraft_type_new']
target = "target"

In [5]:
# converting the categorical columns to categorical
for col in l_categorical_cols:
    flight_data[col] = flight_data[col].astype('category')

In [6]:
flight_data = flight_data[l_numeric_cols + l_categorical_cols + [target] + ['YEAR']]

In [7]:
# getting dummu values for categorical columns
QUARTER_dummies = pd.get_dummies(flight_data['QUARTER'], prefix='QUARTER', drop_first=True)
MONTH_dummies = pd.get_dummies(flight_data['MONTH'], prefix='MONTH', drop_first=True)
DAY_OF_MONTH_dummies = pd.get_dummies(flight_data['DAY_OF_MONTH'], prefix='DAY_OF_MONTH', drop_first=True)
DAY_OF_WEEK_dummies = pd.get_dummies(flight_data['DAY_OF_WEEK'], prefix='DAY_OF_WEEK', drop_first=True)
Aircraft_type_dummies = pd.get_dummies(flight_data['aircraft_type_new'], prefix='Aircraft_type', drop_first=True)

In [8]:
# Aircraft_type_dummies.columns = ['Aircraft_{}'.format(i) for i in range(len(Aircraft_type_dummies.columns))]

In [9]:
flight_data = flight_data.drop(l_categorical_cols, axis=1)

In [10]:
dummy_flight_data = pd.concat([flight_data, QUARTER_dummies, MONTH_dummies,
                               DAY_OF_MONTH_dummies, DAY_OF_WEEK_dummies, Aircraft_type_dummies ], axis=1)

In [11]:
del Aircraft_type_dummies,QUARTER_dummies,MONTH_dummies,DAY_OF_MONTH_dummies,DAY_OF_WEEK_dummies

In [14]:
train = dummy_flight_data[dummy_flight_data['YEAR'] == 2020]
test = dummy_flight_data[dummy_flight_data['YEAR'] == 2019]

In [15]:
del flight_data, dummy_flight_data

In [16]:
logistic_regression = LogisticRegression(multi_class="ovr")
logistic_regression.fit(
    train.drop(["YEAR","target"], axis=1).values,
     train[target]
     )

C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

LogisticRegression(multi_class='ovr')

In [17]:
path = "./models"
filename = "2020train_LR_ovr.mdl"

In [18]:
with open(os.path.join(path, filename), "wb") as f:
    pickle.dump(logistic_regression, f, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
logistic_regression = pickle.load(open("./models/2020train_LR_ovr.mdl",'rb'))

C:\anaconda\envs\python_37\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [13]:
feature_names = dummy_flight_data.columns.drop(["YEAR","target"])

In [14]:
feature_importance = abs(logistic_regression.coef_[0])
feature_importance = 100.0 * (feature_importance / feature_importance.max())

In [15]:
importances = pd.DataFrame(
  pd.Series(dict(zip(feature_names, feature_importance)))
).assign(
  normalized=lambda df: df / df.sum()
).sort_values(
  by="normalized", ascending=False
)

In [16]:
importances[["normalized"]].head(10)

,normalized
MONTH_2,0.070910
QUARTER_4,0.067815
Aircraft_type_Boeing 737,0.061708
QUARTER_2,0.053752
Aircraft_type_Mitsubishi CRJ,0.040773
MONTH_11,0.040590
prcp,0.038558
DEP_HOUR,0.034645
Aircraft_type_Airbus A320,0.033247
DAY_OF_WEEK_5,0.032084


In [20]:
# predicting the test results 
y_pred = logistic_regression.predict(test.drop(["YEAR","target"], axis=1).values)

In [21]:
print(confusion_matrix(test[target],y_pred))

[[6499114       0       0     969]
 [ 903408       0       0     290]
 [ 317781       0       0     162]
 [ 215577       0       0     162]]


In [22]:
print(classification_report(test[target],y_pred))

C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.82      1.00      0.90   6500083
           1       0.00      0.00      0.00    903698
           2       0.00      0.00      0.00    317943
           3       0.10      0.00      0.00    215739

    accuracy                           0.82   7937463
   macro avg       0.23      0.25      0.23   7937463
weighted avg       0.67      0.82      0.74   7937463



C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
print("Testing  set score for model: %f" % logistic_regression.score(test.drop(["YEAR","target"], axis=1).values  , test[target] ))

Testing  set score for model: 0.818810


In [16]:
###### logistic regression multinomial ####
logistic_regression_mul = LogisticRegression(multi_class="multinomial")
logistic_regression_mul.fit(
    train.drop(["YEAR","target"], axis=1).values,
     train[target]
     )

C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(multi_class='multinomial')

In [17]:
path = "./models"
filename = "2020train_LR_mul.mdl"

In [18]:
with open(os.path.join(path, filename), "wb") as f:
    pickle.dump(logistic_regression_mul, f, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
logistic_regression_mul = pickle.load(open("./models/2020train_LR_mul.mdl",'rb'))

C:\anaconda\envs\python_37\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [18]:
feature_importance_mul = abs(logistic_regression_mul.coef_[0])
feature_importance_mul = 100.0 * (feature_importance_mul / feature_importance_mul.max())

In [19]:
importances_mul = pd.DataFrame(
  pd.Series(dict(zip(feature_names, feature_importance_mul)))
).assign(
  normalized=lambda df: df / df.sum()
).sort_values(
  by="normalized", ascending=False
)

In [20]:
importances_mul[["normalized"]].head(10)

,normalized
DEP_HOUR,0.375379
coco,0.114452
wspd,0.060758
temp,0.055337
dwpt,0.045036
prcp,0.029117
MONTH_2,0.026683
rhum,0.021202
QUARTER_4,0.020269
QUARTER_2,0.020048


In [17]:
# predicting the test results 
y_pred = logistic_regression_mul.predict(test.drop(["YEAR","target"], axis=1).values)

In [18]:
print(confusion_matrix(test[target],y_pred))

[[6500083       0       0       0]
 [ 903698       0       0       0]
 [ 317943       0       0       0]
 [ 215739       0       0       0]]


In [19]:
print(classification_report(test[target],y_pred))

C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.82      1.00      0.90   6500083
           1       0.00      0.00      0.00    903698
           2       0.00      0.00      0.00    317943
           3       0.00      0.00      0.00    215739

    accuracy                           0.82   7937463
   macro avg       0.20      0.25      0.23   7937463
weighted avg       0.67      0.82      0.74   7937463



C:\anaconda\envs\python_3_7_p\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
print("Testing  set score for model: %f" % logistic_regression_mul.score(test.drop(["YEAR","target"], axis=1).values  , test[target] ))

Testing  set score for model: 0.818912
